In [11]:
#!/usr/bin/env python3
"""
Worker–Job Matching Program
Consumes outputs from:
1) Worker Assessment program
2) Job Assessment program

Standard library only. Python 3.8+
"""

def get_score(prompt):
    """Get a numeric score between 0 and 100."""
    while True:
        try:
            value = float(input(prompt).strip())
            if 0 <= value <= 100:
                return value
            else:
                print("Please enter a number between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a number between 0 and 100.")


def get_yes_no(prompt):
    """Get a Yes/No answer, returned as boolean."""
    while True:
        value = input(prompt).strip().lower()
        if value in ("yes", "y"):
            return True
        elif value in ("no", "n"):
            return False
        else:
            print("Please enter Yes or No.")


def is_eligible(readiness_score, job_fit_score, risk_score, complexity_score):
    if risk_score > 70 and readiness_score < 50:
        return False
    if complexity_score > job_fit_score + 20:
        return False
    return True


def compute_compatibility(job_fit_score, complexity_score,
                          readiness_score, risk_score,
                          accessibility_score, guidance):
    # Skill vs complexity alignment (40%)
    skill_alignment = max(0, 100 - abs(job_fit_score - complexity_score))
    skill_component = 0.4 * skill_alignment

    # Readiness vs risk alignment (30%)
    readiness_alignment = max(0, 100 - abs(readiness_score - (100 - risk_score)))
    readiness_component = 0.3 * readiness_alignment

    # Accessibility contribution (20%)
    accessibility_component = 0.2 * accessibility_score

    # Guidance bonus (10%)
    guidance_component = 0
    if readiness_score < 60 and guidance:
        guidance_component = 10

    compatibility = (
        skill_component +
        readiness_component +
        accessibility_component +
        guidance_component
    )

    return min(round(compatibility, 2), 100)


def build_explanation(eligible, compatibility, final_score,
                      readiness_score, risk_score,
                      complexity_score, job_fit_score,
                      accessibility_score, guidance):
    if not eligible:
        if risk_score > 70 and readiness_score < 50:
            return (
                "This job was marked unsafe because the job risk is very high "
                "while the worker's readiness is low."
            )
        if complexity_score > job_fit_score + 20:
            return (
                "This job requires skills significantly beyond the worker’s current fit level."
            )
        return "This match did not meet minimum safety or suitability requirements."

    reasons = []

    if abs(job_fit_score - complexity_score) <= 15:
        reasons.append("the worker’s skills align well with the job complexity")
    else:
        reasons.append("there is a noticeable gap between job complexity and worker skills")

    if risk_score <= readiness_score:
        reasons.append("the worker appears ready to handle the job’s risk level")
    else:
        reasons.append("the job risk may be challenging given the worker’s readiness")

    if accessibility_score >= 70:
        reasons.append("the job is highly accessible for the worker")
    elif accessibility_score >= 40:
        reasons.append("the job is moderately accessible")
    else:
        reasons.append("the job has low accessibility")

    if guidance and readiness_score < 60:
        reasons.append("available guidance improves suitability for this worker")

    explanation = "This match was evaluated as follows: " + "; ".join(reasons) + "."

    if final_score >= 75:
        explanation += " Overall, this is a strong and well-balanced match."
    elif final_score >= 55:
        explanation += " Overall, this match may work with some caution or support."
    else:
        explanation += " Overall, this match is weak and may not be ideal."

    return explanation


def main():
    print("\n--- ENTER WORKER ASSESSMENT OUTPUTS ---")
    support_score = get_score("Support Score (0–100): ")
    job_fit_score = get_score("Job Fit Score (0–100): ")
    readiness_score = get_score("Readiness Score (0–100): ")
    overall_urgency = get_score("Overall Urgency (0–100): ")

    print("\n--- ENTER JOB ASSESSMENT OUTPUTS ---")
    complexity_score = get_score("Complexity Score (0–100): ")
    risk_score = get_score("Risk Score (0–100): ")
    accessibility_score = get_score("Accessibility Score (0–100): ")
    allows_unskilled = get_yes_no("Allows Unskilled Workers? (Yes/No): ")
    guidance = get_yes_no("Guidance Provided? (Yes/No): ")

    eligible = is_eligible(
        readiness_score=readiness_score,
        job_fit_score=job_fit_score,
        risk_score=risk_score,
        complexity_score=complexity_score
    )

    if eligible:
        compatibility_score = compute_compatibility(
            job_fit_score=job_fit_score,
            complexity_score=complexity_score,
            readiness_score=readiness_score,
            risk_score=risk_score,
            accessibility_score=accessibility_score,
            guidance=guidance
        )

        final_match_score = round(
            0.5 * compatibility_score +
            0.3 * overall_urgency +
            0.2 * support_score,
            2
        )

        if final_match_score >= 75:
            label = "Strong Match"
        elif final_match_score >= 55:
            label = "Possible Match"
        else:
            label = "Weak Match"
    else:
        compatibility_score = 0
        final_match_score = 0
        label = "Not Suitable / Unsafe"

    explanation = build_explanation(
        eligible=eligible,
        compatibility=compatibility_score,
        final_score=final_match_score,
        readiness_score=readiness_score,
        risk_score=risk_score,
        complexity_score=complexity_score,
        job_fit_score=job_fit_score,
        accessibility_score=accessibility_score,
        guidance=guidance
    )

    print("\n========== MATCH REPORT ==========")
    print(f"Eligibility        : {'Yes' if eligible else 'No'}")
    print(f"Compatibility Score: {compatibility_score}")
    print(f"Final Match Score  : {final_match_score}")
    print(f"Match Label        : {label}")
    print("\nExplanation:")
    print(explanation)
    print("=================================\n")


if __name__ == "__main__":
    main()



--- ENTER WORKER ASSESSMENT OUTPUTS ---


Support Score (0–100):  54
Job Fit Score (0–100):  72
Readiness Score (0–100):  100
Overall Urgency (0–100):  65.5



--- ENTER JOB ASSESSMENT OUTPUTS ---


Complexity Score (0–100):  60
Risk Score (0–100):  85
Accessibility Score (0–100):  61
Allows Unskilled Workers? (Yes/No):  yes
Guidance Provided? (Yes/No):  yes



========== MATCH REPORT ==========
Eligibility        : Yes
Compatibility Score: 51.9
Final Match Score  : 56.4
Match Label        : Possible Match

Explanation:
This match was evaluated as follows: the worker’s skills align well with the job complexity; the worker appears ready to handle the job’s risk level; the job is moderately accessible. Overall, this match may work with some caution or support.

